In [1]:
import os, sys
%pwd

'/config/workspace/research'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    max_words: int
    max_len: int
    batch_size: int
    epochs: int
    validation_split: float

In [4]:
from textClassification.constants import *
from textClassification.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            max_words = params.max_words,
            max_len = params.max_len,
            batch_size = params.batch_size,
            epochs = params.epochs,
            validation_split = params.validation_split
            
        )

        return model_trainer_config

In [6]:
import pandas as pd
import pickle
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import os
import json

2024-03-17 20:41:38.333666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-17 20:41:38.333723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):

        df = pd.read_csv(os.path.join(self.config.data_path,"main_df.csv"))
        df.Sentence=df.Sentence.astype(str)

        x = df['Sentence']
        y = df['Sentiment']

        # Let's split the data into train and test
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 42)

        print(len(x_train),len(y_train))
        print(len(x_test),len(y_test))


        max_words = self.config.max_words
        max_len = self.config.max_len

        tokenizer = Tokenizer(num_words=max_words)
        tokenizer.fit_on_texts(x_train)


        sequences = tokenizer.texts_to_sequences(x_train)
        sequences_matrix = pad_sequences(sequences,maxlen=max_len)
        
        #saving tokenizer
        with open(os.path.join(self.config.root_dir,'tokenizer.pickle'), 'wb') as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


        # Creating model architecture.
        model = Sequential()
        model.add(Embedding(self.config.max_words,100,input_length=self.config.max_len))
        model.add(SpatialDropout1D(0.2))
        model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
        model.add(Dense(1,activation='sigmoid'))
        model.summary()

        model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

        # starting model training
        model.fit(sequences_matrix,y_train,batch_size=self.config.batch_size,epochs = self.config.epochs,validation_split=self.config.validation_split)

        test_sequences = tokenizer.texts_to_sequences(x_test)
        test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

        # Model evaluation
        accr = model.evaluate(test_sequences_matrix,y_test)

        metrics = {"eval": accr}

        with open(os.path.join(self.config.root_dir,'metrics.json'), "w") as file:
            json.dump(metrics, file)


        # Let's save the mdoel.
        model.save(os.path.join(self.config.root_dir,'model.h5'))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-03-17 20:41:40,989: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-17 20:41:40,992: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-17 20:41:40,993: INFO: common: created directory at: artifacts]
[2024-03-17 20:41:40,994: INFO: common: created directory at: artifacts/model_trainer]
4381 4381
1461 1461


2024-03-17 20:41:43.096787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-17 20:41:43.096855: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-17 20:41:43.096889: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c82df065a681): /proc/driver/nvidia/version does not exist
2024-03-17 20:41:43.097225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,080,501
Trainable params: 5,080,501
Non-trainable params: 0
_________________________________________________________________
46/46 [==============================] - 3s 69ms/step - loss: 0.0000e+00 - accuracy: 0.5325
